In [12]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/New_traces/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_New_traces.json"
num_ues_list = [5, 10, 15, 20, 25]
embb_prb_list = [1, 3, 9, 27, 81]
urllc_prb_list = [1, 3, 9, 27, 81]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 2750 training configurations and saved to xapp_dqn_training_configs_New_traces.json


## 20 seconds

In [13]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/Trace_20s/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s.json"
num_ues_list = [5, 10, 15, 20, 25]
embb_prb_list = [1, 3, 9, 27, 81]
urllc_prb_list = [1, 3, 9, 27, 81]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 2750 training configurations and saved to xapp_dqn_training_configs_Trace_20s.json


### Simpler training

In [23]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/Trace_20s/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_13579_all_traces.json"
num_ues_list = [1,3,5,7,9]
#num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 1760 training configurations and saved to xapp_dqn_training_configs_Trace_20s_13579_all_traces.json


In [19]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/Trace_20s/simple_setup/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_simpler.json"
#num_ues_list = [1,3,5,7,9]
num_ues_list = [1]
embb_prb_list = [5]
urllc_prb_list = [5]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None


# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 1 training configurations and saved to xapp_dqn_training_configs_Trace_20s_simpler.json


In [22]:
import json

INPUT = "../assets/episodes/gym_from_training_config_1ms_Trace_20s_simpler.json"
OUTPUT = "../assets/episodes/gym_repeated_episodes.json"
REPEAT = 200  # how many episodes you want

with open(INPUT, "r") as f:
    cfg = json.load(f)

base_ep = cfg["episodes"][0]

episodes = []
for i in range(REPEAT):
    ep = base_ep.copy()
    ep["id"] = f"cfg_{i:05d}"
    episodes.append(ep)

with open(OUTPUT, "w") as f:
    json.dump({"episodes": episodes}, f, indent=2)

print("Done!")


Done!


## Config for test

##### Number of UEs

In [30]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_UE_test/Trace_20s/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_12468_all_traces_test_8.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [8]
embb_prb_list = [5]
urllc_prb_list = [5]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 22 training configurations and saved to xapp_dqn_training_configs_Trace_20s_12468_all_traces_test_8.json


#### Packet size std

In [31]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std1/std1_2/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std1_2.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std1_2.json


In [32]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std1/std1_4/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std1_4.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std1_4.json


In [33]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std1/std1_6/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std1_6.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std1_6.json


In [34]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std1/std1_8/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std1_8.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std1_8.json


In [35]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std1/std1_10/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std1_10.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std1_10.json


In [36]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std1/std1_12/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std1_12.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std1_12.json


In [37]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std1/std1_14/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std1_14.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std1_14.json


##### std2

In [38]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std2/std2_0.2/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std2_0.2.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std2_0.2.json


In [39]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std2/std2_0.4/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std2_0.4.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std2_0.4.json


In [41]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std2/std2_0.6/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std2_0.6.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std2_0.6.json


In [40]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/std2/std2_0.8/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_std2_0.8.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_std2_0.8.json


#### Transition matrix

##### Alpha

In [42]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/alpha/alpha_0.1/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_alpha_0.1.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_alpha_0.1.json


In [43]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/alpha/alpha_0.6/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_alpha_0.6.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_alpha_0.6.json


In [44]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/alpha/alpha_1.1/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_alpha_1.1.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_alpha_1.1.json


In [45]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/alpha/alpha_1.6/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_alpha_1.6.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_alpha_1.6.json


In [46]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/alpha/alpha_2.1/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_alpha_2.1.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_alpha_2.1.json


##### beta

In [47]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/beta/beta_0.1/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_beta_0.1.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_beta_0.1.json


In [48]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/beta/beta_0.6/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_beta_0.6.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_beta_0.6.json


In [49]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/beta/beta_1.1/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_beta_1.1.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_beta_1.1.json


In [50]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/beta/beta_1.6/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_beta_1.6.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_beta_1.6.json


In [51]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC_test/beta/beta_2.1/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_Trace_20s_1_test_beta_2.1.json"
#num_ues_list = [1,2,4,6,8]
num_ues_list = [1]
embb_prb_list = [5, 7, 9, 11]
urllc_prb_list = [5, 7, 9, 11]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")


✅ Generated 16 training configurations and saved to xapp_dqn_training_configs_Trace_20s_1_test_beta_2.1.json


In [ ]:
import os
import json
import re
import itertools

# =====================================
# Configuration
# =====================================
#TRACE_DIR = "./Unified_CMTC/traces/aligned/"  # folder where trace CSVs are saved
TRACE_DIR = "./Unified_CMTC/New_traces/traces/aligned/"
#OUTPUT_JSON = "xapp_dqn_training_configs.json"
OUTPUT_JSON = "xapp_dqn_training_configs_New_traces.json"
num_ues_list = [2, 4, 8, 16, 20]
embb_prb_list = [3]
urllc_prb_list = [3]

# =====================================
# Helper to categorize trace files
# =====================================
def parse_trace_file(filename):
    """
    Extract trace type and value from filenames like:
    eMBB_M2_alpha_0.5_aligned_trace_sec.csv
    URLLC_M1_std1_11_aligned_trace_sec.csv
    """
    match = re.search(r"(std1|std2|alpha|beta)_([\d.]+)", filename)
    if match:
        return match.group(1), float(match.group(2))
    return None, None

# =====================================
# Build dictionary of available traces
# =====================================
traces = {"embb": {}, "urllc": {}}
for fname in os.listdir(TRACE_DIR):
    if not fname.endswith(".csv"):
        continue

    ttype, tval = parse_trace_file(fname)
    if ttype is None:
        continue

    if fname.startswith("eMBB"):
        traces["embb"].setdefault(ttype, {})[tval] = fname
    elif fname.startswith("URLLC"):
        traces["urllc"].setdefault(ttype, {})[tval] = fname

# =====================================
# Build all config combinations
# =====================================
configs = []
for num_ues, embb_prb, urllc_prb in itertools.product(num_ues_list, embb_prb_list, urllc_prb_list):
    for trace_type, embb_vals in traces["embb"].items():
        if trace_type not in traces["urllc"]:
            continue  # skip if no matching URLLC trace type

        for val, embb_file in embb_vals.items():
            urllc_file = traces["urllc"][trace_type].get(val)
            if urllc_file is None:
                continue  # skip if mismatch

            configs.append({
                "num_ues_per_slice": num_ues,
                "embb_default_prb": embb_prb,
                "urllc_default_prb": urllc_prb,
                "trace_type": trace_type,
                "trace_value": val,
                "trace_files": {
                    "embb": embb_file,
                    "urllc": urllc_file
                }
            })

# =====================================
# Save configs
# =====================================
with open(OUTPUT_JSON, "w") as f:
    json.dump(configs, f, indent=2)

print(f"✅ Generated {len(configs)} training configurations and saved to {OUTPUT_JSON}")
